# RAG 챗봇 개발 (OpenAI API, Langchain, Streamlit)

## 폴더 구조

```
ROOT/
- data/ --> 문서가 들어있는 폴더
- .env --> OpenAI API 키가 저장되어 있는 파일.
- db/ --> 벡터DB (처음에는 샘플 문서가 저장)
- utils.py --> RAG 챗봇 로직 코드
- rag.py --> 챗봇 (과거기억 못합) UI 코드
- history_rag.py --> 챗봇 (과거기억함) UI 코드
- RAG 챗봇 개발 (OpenAI API, Langchain, Streamlit).ipynb --> 현재 노트북 파일
```

## 사용 툴

- OpenAI API
    - 임베딩 생성 모델과 LLM을 제공합니다.
    - 모델은 OpenAI의 서버에 올려져 있으므로 민감한 문서나 내용을 넘기지 않도록 주의가 필요합니다.
    - 유료라서 OpenAI API 계정에 돈을 충전해야 합니다. $1면 충분합니다. ⚠ ChatGPT Plus와는 다릅니다. ⚠
- Langchain
    - LLM 앱 개발을 위한 프레임워크입니다.
- ChromaDB
    - 문서를 저장할 오픈소스 벡터DB입니다.
- Streamlit
    - 챗봇 UI를 생성하기 위해 사용할 프레임워크입니다.



## 환경설정

구글 colab CPU 환경에서 실행하는 것을 가정합니다.

1. .env 파일에 `OPENAI_API_KEY`가 있어야 합니다.
2. 구글 드라이브를 마운트합니다. (밑 코드 실행)
3. 필요한 패키지를 다운받습니다. (밑 코드 실행)

구글 드라이브를 연결합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


`ROOT`변수에 `.env` 파일이 들어있는 폴더 경로를 입력합니다.

In [2]:
ROOT = "/content/drive/MyDrive/AI대회_참고자료" # 수정필요

`ROOT`에 입력한 경로로 이동합니다.

In [6]:
%cd $ROOT

/content/drive/MyDrive/AI대회_참고자료


필요한 패키지를 다운받습니다.

In [3]:
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma pypdf python-dotenv langchain-openai streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [5]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.185s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [9]:
import os
from dotenv import load_dotenv

# .env 파일에 저장된 환경변수를 불러옵니다.
load_dotenv()

True

## 문서를 벡터 DB에 저장하기

`ROOT/data` 폴더에 PDF 문서들이 저장되어 있다고 가정합니다.

다른 경우는 [공식 문서](https://python.langchain.com/v0.1/docs/integrations/document_loaders/)를 참고하면 됩니다.
- PDF가 아닌 문서를 다루는 경우 eg. [MS word](https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/)
- 웹에서 문서를 다운받는 경우

In [10]:
pdf_folder_path = "./data"

In [24]:
# 폴더 속 모든 PDF 파일 로드
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()
print(f"{len(docs)} 페이지 로드")

문서를 1000자(200자는 중복)로 자르고 임베딩해서 벡터DB에 저장합니다.

`persist_directory`에 벡터DB를 저장할 위치를 입력합니다. 예시에서는 `./db` 폴더로 설정되어 있습니다.

In [31]:
### OpenAI Embedding 사용해서 문서를 임베딩 후 Chroma 벡터DB에 저장.
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embedder = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedder,
    persist_directory="./db"
)
retriever = vectorstore.as_retriever()

## Streamlit으로 챗봇 UI 구현하기

### 방법

1. 미리 모든 문서를 청크로 분리하고 임베딩해서 벡터DB에 저장합니다.
2. 사용자한테 질문을 입력받습니다.
3. 질문과 비슷한 문서 청크를 벡터DB에서 찾아 가져옵니다.
4. 가져온 문서 청크를 기반으로 사용자 질문에 대한 답변을 LLM이 생성합니다.

streamlit으로 `rag.py` 코드를 실행하면 챗봇 앱을 시작합니다.

마지막 줄에 있는 링크로 들어갑니다. `Tunnel Password`는 첫 번째 줄에 출력된 ip 주소 입니다.

In [48]:
!streamlit run rag.py \
&>/content/logs.txt \
& npx localtunnel --port 8501 \
& curl ipv4.icanhazip.com

34.74.109.162
npx: installed 22 in 2.322s
your url is: https://free-tips-lie.loca.lt


## Streamlit으로 챗봇 UI 구현하기 (과거 질문답변 기억)

### 방법

1. 미리 모든 문서를 청크로 분리하고 임베딩해서 벡터DB에 저장합니다.
2. 사용자한테 질문을 입력받습니다.
3. 과거 질문답변 내역 없이도 질문에 답변할 수 있도록 새로운 질문으로 만들어달라고 LLM에 요청합니다.
3. 새로운 질문과 비슷한 문서 청크를 벡터DB에서 찾아 가져옵니다.
4. 가져온 문서 청크와 과거 질문답변 내역을 기반으로 사용자 질문에 대한 답변을 LLM이 생성합니다.

streamlit으로 `history_rag.py` 코드를 실행하면 챗봇 앱을 시작합니다.

마지막 줄에 있는 링크로 들어갑니다. `Tunnel Password`는 첫 번째 줄에 출력된 ip 주소 입니다.

In [51]:
!streamlit run history_rag.py \
&>/content/logs.txt \
& npx localtunnel --port 8501 \
& curl ipv4.icanhazip.com

34.74.109.162
npx: installed 22 in 2.555s
your url is: https://tall-beers-laugh.loca.lt


## (부록) 코드 파일 작성

In [ ]:
%%writefile utils.py
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# LLM 로드
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

vectorstore = Chroma(
    persist_directory="./db",
    embedding_function=OpenAIEmbeddings())
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

def get_rag_chain():

    # template = """제공된 맥락을 사용하여 질문에 답변하십시오.
    # 답을 모르겠다면 모른다고 말하고, 답을 지어내지 마십시오.
    # 답변을 간결하게 작성하십시오.

    # {context}

    # 질문: {question}

    # 답변:"""

    template = """You are an assistant for question-answering tasks. \
    Use the following pieces of retrieved context to answer the question. \
    If you don't know the answer, just say that you don't know. \

    {context}

    질문: {question}

    답변:"""

    custom_rag_prompt = PromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

def get_history_rag_chain():

    contextualize_q_system_prompt = """Given a chat history and the latest user question \
    which might reference context in the chat history, formulate a standalone question \
    which can be understood without the chat history. Do NOT answer the question, \
    just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    qa_system_prompt = """You are an assistant for question-answering tasks. \
    Use the following pieces of retrieved context to answer the question. \
    If you don't know the answer, just say that you don't know. \

    {context}"""

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", qa_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain

Overwriting utils.py


In [ ]:
%%writefile rag.py
import streamlit as st
from utils import get_rag_chain

rag_chain = get_rag_chain()

st.title("📝 PDF 기반 QA 챗봇")

if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "질문에 답변해드립니다."}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    response = st.chat_message("assistant").write_stream(rag_chain.stream(prompt))
    st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting rag.py


In [50]:
%%writefile history_rag.py
import streamlit as st
from langchain_community.chat_message_histories import (
    StreamlitChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from utils import get_history_rag_chain

rag_chain = get_history_rag_chain()

history = StreamlitChatMessageHistory(key="test")

if len(history.messages) == 0 or st.sidebar.button("초기화"):
    history.clear()
    history.add_ai_message("문서 내용에 대해 질문하면 답변해드립니다.")


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    # get_session_history,
    lambda session_id: history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

for msg in history.messages:
    st.chat_message(msg.type).write(msg.content)

if prompt := st.chat_input():
    st.chat_message("human").write(prompt)

    # As usual, new messages are added to StreamlitChatMessageHistory when the Chain is called.
    config = {"configurable": {"session_id": "test"}}
    with st.spinner("답변 확인 중..."):
        response = conversational_rag_chain.invoke({"input": prompt}, config)
    st.chat_message("ai").write(response["answer"])

Overwriting history_rag.py


# 코드 설명

## RAG 챗봇 (과거 질문답변 기억 못 함)

미리 만들어 놓은 벡터DB를 불러옵니다. 사용자 질문과 유사한 문서를 찾아올 `retriever`도 설정합니다.

벡터DB는 `db/` 폴더에 저장되어 있다고 가정합니다.

In [52]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma(
    persist_directory="./db",
    embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

답변을 생성하는데 사용할 LLM을 불러옵니다.

OpenAI API를 사용해서 gpt-3.5 모델을 불러옵니다.

In [53]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

LLM에게 넘겨줄 프롬프트 형식입니다.

- `context`: 문맥 정보를 줄 문서가 들어갑니다.
- `question`: 사용자가 입력한 질문이 들어갑니다.


In [54]:
from langchain_core.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \

{context}

질문: {question}

답변:"""

custom_rag_prompt = PromptTemplate.from_template(template)

- 사용자의 입력을 받으면
- `retriever | format_docs`: 벡터DB에서 문서 가져오기 | 가져온 문서 합치기
- `custom_rag_prompt`: 프롬프트 만들기
- `llm`: LLM으로 생성하기
- `StrOutputParser()`: 출력하기

의 단계를 연결해서 `rag_chain`을 만듭니다.

In [55]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

답변을 잘 하는지 확인해 봅시다.

In [57]:
print(rag_chain.invoke("문서의 목차를 알려줘"))

- 목차
- 서론
  - 본 백서 작성의 배경
    - LLM을 왜 평가하는가?
  - 백서의 대상 독자
  - LLM 평가의 프레임워크와 용어 정리
- 언어 모델 평가의 전체 모습
  - 본 백서에서 제안하는 체계
- LLM 모델의 평가 방법
  - What to evaluate: 평가할 측면
    - 기초 언어 능력
      - 언어 이해
      - 자연어 추론
    - 응용 능력
      - 지식·질의응답
      - 추출
      - 수학적 추론
      - 독해
      - 번역
      - 표현
    - 평가 데이터의 유출
    - 전문 능력
    - 정렬
      - 제어성
      - 윤리·도덕
      - 유해성
      - 편향
      - 진실성
      - 견고성
  - How to evaluate: 평가 방법
    - 기반 모델의 평가 vs 파인튜닝된 모델의 평가
    - 평가의 단계
      - 프롬프트 설계
      - Few-shot vs Zero-shot
      - 로그 우도 선택 방식과 직접 생성 방식
      - 출력의 후처리와 정답의 일의성 담보
    - 평가 산출 방법
      - 일반적인 벤치마크에 의한 평가
- 본 백서에서 제안하는 체계
  - 범용 언어 성능
  - AI 거버넌스 도메인 특화 성능
  - 응용 능력
    - 표현
    - 번역
    - 독해
    - 수학적 배경 추출
    - 지식, 깊은 응답
  - 자연어 추론 자연어 이해
    - 기초적 언어 능력
    - 전문 능력
  - 법률
  - 의료
  - 금융
  - 프로그래밍
  - 그 외 기타
  - 얼라인먼트
    - 제어성
    - 윤리·도덕
    - 유해성
    - 편향성
    - 진실성
    - 견고성
  - 시스템 성능
    - 추론 속도
    - 지론 호출
  - 시큐리티
- 그림 4: 본 백서에서 제안하는 LLM 평가의 전체 모습과 체계


## RAG 챗봇 (과거 질문답변 기억)

벡터DB, retriever, LLM을 불러오는 것은 위와 동일합니다.

In [58]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma(
    persist_directory="./db",
    embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [59]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

문서를 벡터DB에서 가져올 때는 질문만 보기 때문에 해당 질문이 과거의 질문답변 내용을 반영하고 있도록 수정해야 합니다.

LLM을 사용해서 질문을 수정하고, 수정된 질문으로 벡터DB에서 유사한 문서를 가져오는 단계입니다.

In [60]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

사용자 질문, 문맥이 되는 문서, 과거의 질문답변 내용을 입력하고 LLM으로 질문에 대한 답변을 생성하는 단계입니다.

In [65]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

위 단계를 연결해서 `history_rag_chain`을 만듭니다

1. 미리 모든 문서를 청크로 분리하고 임베딩해서 벡터DB에 저장합니다.
2. 사용자한테 질문을 입력받습니다.
3. 과거 질문답변 내역 없이도 질문에 답변할 수 있도록 새로운 질문으로 만들어달라고 LLM에 요청합니다.
3. 새로운 질문과 비슷한 문서 청크를 벡터DB에서 찾아 가져옵니다.
4. 가져온 문서 청크와 과거 질문답변 내역을 기반으로 사용자 질문에 대한 답변을 LLM이 생성합니다.

In [66]:
from langchain.chains import create_retrieval_chain

history_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

임시로 과거 질문답변을 저장할 수 있는 공간을 만들기 위해 `ChatMessageHistory`를 사용합니다.

(앱에서는 `StreamlitChatMessageHistory`을 사용합니다.)

`RunnableWithMessageHistory`로 질문을 할 때 자동으로 질문답변 내역을 기록합니다.

In [72]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    history_rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

잘 답변하는지 확인해 봅시다.

In [87]:
get_session_history("test").clear() # 과거내역을 삭제합니다.
print("######## 첫 번째 질문: 문서의 목차를 알려줘 ############")
response = conversational_rag_chain.invoke(
    {"input": "문서의 목차를 알려줘"},
    config={
        "configurable": {"session_id": "test"}
    },
)["answer"]
print(response)
print()

print("######## 두 번째 질문: 첫 번째 항목에 대해 설명해줘. ############")
response = conversational_rag_chain.invoke(
    {"input": "첫 번째 항목에 대해 설명해줘."},
    config={
        "configurable": {"session_id": "test"}
    },
)["answer"]
print(response)

######## 첫 번째 질문: 문서의 목차를 알려줘 ############
본 문서의 목차는 다음과 같습니다:
1. 서론
   - 본 백서 작성의 배경
   - LLM을 왜 평가하는가?
   - 백서의 대상 독자
   - LLM 평가의 프레임워크와 용어 정리
2. 언어 모델 평가의 전체 모습
   - 본 백서에서 제안하는 체계
3. LLM 모델의 평가 방법
   - What to evaluate: 평가할 측면
   - How to evaluate: 평가 방법
      - 기반 모델의 평가 vs 파인튜닝된 모델의 평가
      - 평가의 단계
      - 평가 산출 방법
4. 그림 4: 본 백서에서 제안하는 LLM 평가의 전체 모습과 체계

######## 두 번째 질문: 첫 번째 항목에 대해 설명해줘. ############
첫 번째 항목은 "서론"입니다. 이 항목에서는 본 백서 작성의 배경, LLM을 평가하는 이유, 대상 독자, 그리고 LLM 평가를 위한 프레임워크와 용어에 대한 정리가 포함됩니다. 여기서는 왜 LLM을 평가해야 하는지와 이를 어떤 관점에서 다룰 것인지에 대한 내용이 다루어집니다.


과거 질문답변이 저장되는 것을 볼 수 있습니다.

In [88]:
print(get_session_history("test"))

Human: 문서의 목차를 알려줘
AI: 본 문서의 목차는 다음과 같습니다:
1. 서론
   - 본 백서 작성의 배경
   - LLM을 왜 평가하는가?
   - 백서의 대상 독자
   - LLM 평가의 프레임워크와 용어 정리
2. 언어 모델 평가의 전체 모습
   - 본 백서에서 제안하는 체계
3. LLM 모델의 평가 방법
   - What to evaluate: 평가할 측면
   - How to evaluate: 평가 방법
      - 기반 모델의 평가 vs 파인튜닝된 모델의 평가
      - 평가의 단계
      - 평가 산출 방법
4. 그림 4: 본 백서에서 제안하는 LLM 평가의 전체 모습과 체계
Human: 첫 번째 항목에 대해 설명해줘.
AI: 첫 번째 항목은 "서론"입니다. 이 항목에서는 본 백서 작성의 배경, LLM을 평가하는 이유, 대상 독자, 그리고 LLM 평가를 위한 프레임워크와 용어에 대한 정리가 포함됩니다. 여기서는 왜 LLM을 평가해야 하는지와 이를 어떤 관점에서 다룰 것인지에 대한 내용이 다루어집니다.


반면 과거 질문답변을 기억하지 않는 챗봇은 엉뚱한 답을 낼 수 있습니다.



In [89]:
print("######## 첫 번째 질문: 문서의 목차를 알려줘 ############")
print(rag_chain.invoke("문서의 목차를 알려줘"))
print()
print("######## 두 번째 질문: 첫 번째 항목에 대해 설명해줘. ############")
print(rag_chain.invoke("첫 번째 항목에 대해 설명해줘."))

######## 첫 번째 질문: 문서의 목차를 알려줘 ############
- 목차
- 목차
- 서론
  - 본 백서 작성의 배경
    - LLM을 왜 평가하는가?
  - 백서의 대상 독자
  - LLM 평가의 프레임워크와 용어 정리
- 언어 모델 평가의 전체 모습
  - 본 백서에서 제안하는 체계
- LLM 모델의 평가 방법
  - What to evaluate: 평가할 측면
    - 기초 언어 능력
      - 언어 이해(Natural Language Understanding)
      - 자연어 추론(Natural Language Inference)
    - 응용 능력(Application Skills)
      - 지식·질의응답(Knowledge / Question Answering)
      - 추출(Extraction)
      - 수학적 추론(Mathematical Reasoning)
      - 독해(Reading Comprehension)
      - 번역(Translation)
      - 표현(Expression)
    - 평가 데이터의 유출
    - 전문 능력
    - 정렬(Alignment)
      - 제어성(Controllability)
      - 윤리·도덕(Ethics / Morality)
      - 유해성(Toxicity)
      - 편향(Bias)
      - 진실성(Truthfulness)
      - 견고성(Robustness)
  - How to evaluate: 평가 방법
    - 기반 모델의 평가 vs 파인튜닝된 모델의 평가
    - 평가의 단계
      - 프롬프트 설계
      - Few-shot vs Zero-shot
      - 로그 우도 선택 방식과 직접 생성 방식
      - 출력의 후처리와 정답의 일의성 담보
    - 평가 산출 방법
      - 일반적인 벤치마크에 의한 평가
- 본 백서에서 제안하는 체계
  - 범용 언어 성능
  - AI 거버넌스도

# 챗봇 답변 성능 높이는 방법

- LLM에 입력할 프롬프트 개선
- 문서를 벡터DB에 넣기 전에 청크로 나누는 방법 변경 (chunk_size, chunk_overlap)
- 문서 가져오는(retrieve) 방법 개선
- 문서 가져온(retrieve) 후 순위 조정(reranking) 단계 추가
- 더 성능 좋은 모델로 변경 (gpt4)